<a href="https://colab.research.google.com/github/Launcha10/sports_entertaiment_topic_analysis/blob/kawamura/sports_randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!apt install aptitude

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [3]:
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y

mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.5)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.3)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.5)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz

In [4]:
!pip install mecab-python3==0.7

In [0]:
# -*- coding: utf-8 -*-
import os
import sys
import re
from gensim import corpora, matutils
import MeCab

DATA_DIR_PATH = '/content/drive/My Drive/sports_text/'
DICTIONARY_FILE_NAME = 'sportsdic.txt'
mecab = MeCab.Tagger('Owakati')


def get_class_id(file_name):
    '''
    ファイル名から、クラスIDを決定する。
    学習データを作るときに使っています。
    '''
    #print("file_nameだよ："+file_name)
    dir_list = get_dir_list()
    #print("dir_listだよ：")
    #print(dir_list)
    
    dir_name = next(filter(lambda x: x in file_name, dir_list),None)
    
    #print("dir_nameだよ：")
    #print(dir_name)
    if dir_name:
        return dir_list.index(dir_name)
    return None
def get_dir_list():
    '''
    ライブドアコーパスが./text/ の下にカテゴリ別にあるからそのカテゴリ一覧をとってるだけ
    '''
    tmp = os.listdir(DATA_DIR_PATH)
    #print(tmp)
    #print("----------------------")
    if tmp is None:
        return None
    return sorted([x for x in tmp if os.path.isdir(DATA_DIR_PATH + x)])


def get_file_content(file_path):
    '''
    1つの記事を読み込み
    '''
    with open(file_path, encoding='utf-8') as f:
        return ''.join(f.readlines()[1:])  # ライブドアコーパスが3行目から本文はじまってるから


def tokenize(text):
    '''
    とりあえず形態素解析して名詞だけ取り出す感じにしてる
    '''
    node = mecab.parseToNode(text)
    while node:
        if node.feature.split(',')[0] == '名詞':
            yield node.surface.lower()
        node = node.next


def check_stopwords(word):
    '''
    ストップワードだったらTrueを返す
    '''
    if re.search(r'^[0-9]+$', word):  # 数字だけ
        return True
    return False


def get_words(contents):
    '''
    記事群のdictについて、形態素解析して返す
    '''
    ret = []
    for k, content in contents.items():
        ret.append(get_words_main(content))
    return ret


def get_words_main(content):
    '''
    一つの記事を形態素解析して返す
    '''
    return [token for token in tokenize(content) if not check_stopwords(token)]


def filter_dictionary(dictionary):
    '''
    低頻度と高頻度のワードを除く感じで
    '''
    dictionary.filter_extremes(no_below=2, no_above=0.5)  # この数字はあとで変えるかも
    return dictionary


def get_contents():
    '''
    livedoorニュースのすべての記事をdictでまとめておく
    '''
    dir_list = get_dir_list()
    #print(dir_list)
    #print("-------------------------")

    if dir_list is None:
        return None

    ret = {}
    for dir_name in dir_list:
        file_list = os.listdir(DATA_DIR_PATH + dir_name)
        #print(len(file_list))
        #print("--------------------")

        if file_list is None:
            continue
        for file_name in file_list:
            #if dir_name in file_name:  # LICENSE.txt とかを除くためです。。
                ret[file_name] = get_file_content(DATA_DIR_PATH + dir_name + '/' + file_name)
        #print(len(ret))    
        #print("-------------------")

    return ret


def get_vector(dictionary, content):
    '''
    ある記事の特徴語カウント
    '''
    a = get_words_main(content)
    
    #print(a)
    tmp = dictionary.doc2bow(get_words_main(content))
    dense = list(matutils.corpus2dense([tmp], num_terms=len(dictionary)).T[0])
    #print(dense)
    return dense


def get_dictionary(create_flg=False, file_name=DICTIONARY_FILE_NAME):
    '''
    辞書を作る
    '''
    if create_flg or not os.path.exists(file_name):
        # データ読み込み
        contents = get_contents()
        
        # 形態素解析して名詞だけ取り出す
        words = get_words(contents)
        #print("wordsだよ")
        #print(words)
        # 辞書作成、そのあとフィルタかける
        dictionary = filter_dictionary(corpora.Dictionary(words))
        #print("dicだよ")
        #print(dictionary.token2id)
        # 保存しておく
        if file_name is None:
            sys.exit()
        dictionary.save_as_text(file_name)

    else:
        # 通常はファイルから読み込むだけにする
        dictionary = corpora.Dictionary.load_from_text(file_name)

    return dictionary


if __name__ == '__main__':
    get_dictionary(create_flg=True)
    




In [6]:
#scikit-learnのインストール
pip install scikit-learn

In [7]:
import sklearn

print(sklearn.__version__)

0.22.1


In [33]:
# -*- coding: utf-8 -*-
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
#import corpus
# 不要な警告を非表示にする
import warnings
warnings.filterwarnings('ignore')


def main():
    # 辞書の読み込み
    dictionary = get_dictionary(create_flg=False)
    #print("辞書")
    #print(dictionary.token2id)
    #print("--------------------------------------------")
    # 記事の読み込み
    contents = get_contents()

    # 特徴抽出
    data_train = []
    label_train = []

    for file_name, content in contents.items():
        data_train.append(get_vector(dictionary, content))
        label_train.append(get_class_id(file_name))
        
      
        
    #print(data_train)
    #print(label_train)

    # 分類器
    estimator = RandomForestClassifier()

    # 学習
    estimator.fit(data_train, label_train)

    # 学習したデータを予測にかけてみる（ズルなので正答率高くないとおかしい）
    print("==== 学習データと予測データが一緒の場合")
    print(estimator.score(data_train, label_train))

    # 学習データと試験データに分けてみる
    data_train_s, data_test_s, label_train_s, label_test_s = train_test_split(data_train, label_train, test_size=0.5)

    # 分類器をもう一度定義
    estimator2 = RandomForestClassifier()

    # 学習
    estimator2.fit(data_train_s, label_train_s)
    print("ラベル")
    print("0:baseball(100)")
    print("1:basket(30)")
    print("2:golf(95)")
    print("3:judo(40)")
    print("4:kendo(18)")
    print("5:rugby(28)")
    print("6:soccer(100)")
    print("7:sumo(62)")
    print("8:tennis(21)")
    print("9:volley(11)")
    #print(len(data_train_s))
    print("--------------------------")
    print("ある文書の特徴ベクトル" , data_train_s[0])
    print("ラベル予想：" , label_train_s[0])
    print("--------------------------")
    print("==== 学習データと予測データが違う場合の正解率")
    print(estimator2.score(data_test_s, label_test_s))

        # 予測
    #label_predict = estimator.predict(data_train_s)
    #print(label_predict)

    
if __name__ == '__main__':
    main()

==== 学習データと予測データが一緒の場合
0.9920792079207921
ラベル
0:baseball(100)
1:basket(30)
2:golf(95)
3:judo(40)
4:kendo(18)
5:rugby(28)
6:soccer(100)
7:sumo(62)
8:tennis(21)
9:volley(11)
--------------------------
ある文書の特徴ベクトル [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 3.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.0, 15.0, 1.0, 42.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 8.0, 0.0, 0.0, 0.0, 0.0, 3.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 6.0, 15.0, 0.0, 3.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 0.0, 3.0, 0.0, 1.0, 21.0, 0.0, 0.0, 4.0, 6.0, 2.0, 5.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 3.0, 0.0, 0.0, 12.0, 19.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 15.0, 0.0, 3.0, 15.0, 0.0, 6.0, 0.0, 2.0, 0.0, 0.0, 0.0, 4.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15

In [0]:
# -*- coding: utf-8 -*-
import os
import sys
import re
from gensim import corpora, matutils
import MeCab

DATA_DIR_PATH = '/content/drive/My Drive/text/'
DICTIONARY_FILE_NAME = 'livedoordic.txt'
mecab = MeCab.Tagger('Owakati')


def get_class_id(file_name):
    '''
    ファイル名から、クラスIDを決定する。
    学習データを作るときに使っています。
    '''
    dir_list = get_dir_list()
    dir_name = next(filter(lambda x: x in file_name, dir_list), None)
    if dir_name:
        return dir_list.index(dir_name)
    return None
def get_dir_list():
    '''
    ライブドアコーパスが./text/ の下にカテゴリ別にあるからそのカテゴリ一覧をとってるだけ
    '''
    tmp = os.listdir(DATA_DIR_PATH)
    if tmp is None:
        return None
    return sorted([x for x in tmp if os.path.isdir(DATA_DIR_PATH + x)])


def get_file_content(file_path):
    '''
    1つの記事を読み込み
    '''
    with open(file_path, encoding='utf-8') as f:
        return ''.join(f.readlines()[2:])  # ライブドアコーパスが3行目から本文はじまってるから


def tokenize(text):
    '''
    とりあえず形態素解析して名詞だけ取り出す感じにしてる
    '''
    node = mecab.parseToNode(text)
    while node:
        if node.feature.split(',')[0] == '名詞':
            yield node.surface.lower()
        node = node.next


def check_stopwords(word):
    '''
    ストップワードだったらTrueを返す
    '''
    if re.search(r'^[0-9]+$', word):  # 数字だけ
        return True
    return False


def get_words(contents):
    '''
    記事群のdictについて、形態素解析して返す
    '''
    ret = []
    for k, content in contents.items():
        ret.append(get_words_main(content))
    return ret


def get_words_main(content):
    '''
    一つの記事を形態素解析して返す
    '''
    return [token for token in tokenize(content) if not check_stopwords(token)]


def filter_dictionary(dictionary):
    '''
    低頻度と高頻度のワードを除く感じで
    '''
    dictionary.filter_extremes(no_below=20, no_above=0.3)  # この数字はあとで変えるかも
    return dictionary


def get_contents():
    '''
    livedoorニュースのすべての記事をdictでまとめておく
    '''
    dir_list = get_dir_list()

    if dir_list is None:
        return None

    ret = {}
    for dir_name in dir_list:
        file_list = os.listdir(DATA_DIR_PATH + dir_name)

        if file_list is None:
            continue
        for file_name in file_list:
            if dir_name in file_name:  # LICENSE.txt とかを除くためです。。
                ret[file_name] = get_file_content(DATA_DIR_PATH + dir_name + '/' + file_name)

    return ret


def get_vector(dictionary, content):
    '''
    ある記事の特徴語カウント
    '''
    tmp = dictionary.doc2bow(get_words_main(content))
    dense = list(matutils.corpus2dense([tmp], num_terms=len(dictionary)).T[0])
    return dense


def get_dictionary(create_flg=False, file_name=DICTIONARY_FILE_NAME):
    '''
    辞書を作る
    '''
    if create_flg or not os.path.exists(file_name):
        # データ読み込み
        contents = get_contents()
        # 形態素解析して名詞だけ取り出す
        words = get_words(contents)
        # 辞書作成、そのあとフィルタかける
        dictionary = filter_dictionary(corpora.Dictionary(words))
        # 保存しておく
        if file_name is None:
            sys.exit()
        dictionary.save_as_text(file_name)

    else:
        # 通常はファイルから読み込むだけにする
        dictionary = corpora.Dictionary.load_from_text(file_name)

    return dictionary


if __name__ == '__main__':
    get_dictionary(create_flg=True)


